In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from xgboost import XGBClassifier
import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter
import joblib
import numpy as np
import os

In [2]:


# data = pd.read_csv("csv/walk.csv",index_col=False)
# encode = {'<40': 0, '>40': 1}
# encode = {'male': 0, 'female': 1}
# X = data.loc[data['id'].isin([1,2])]
# X_train = X.drop(['id','gender','age'],axis=1)

# X_train = preprocessing.scale(X_train)
# y_train = X['age']#.map(encode)


# X2 = data.loc[data['id'].isin([10])]
# X_test = X2.drop(['id','gender','age'],axis=1)

# X_test = preprocessing.scale(X_test)
# y_test = X2['gender'].map(encode)

data = pd.read_csv("csv/walk.csv",index_col=False)
# X = data[["r_hip_ang", "r_knee_ang", "r_ankle_ang", "r_elbow_ang", "r_shoulder_ang",]]
#         # "r_hip_vel", "r_knee_vel", "r_ankle_vel", "r_elbow_vel", "r_shoulder_vel",
#         # "r_hip_accel", "r_knee_accel", "r_ankle_accel", "r_elbow_accel", "r_shoulder_accel"]]
X = data.drop(['gender', "id",'age'], axis=1)

# encode = {'male': 0, 'female': 1}
encode = {'<40': 0, '>40': 1}
y = data['age'].map(encode)
X  = preprocessing.scale(X)

# oversample = SMOTE()
# X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)


ModuleNotFoundError: No module named 'imblearn'

In [10]:
# RANDOM FOREST

# from joblib import Parallel, delayed
import joblib
import numpy as np

randomforest_from_joblib = joblib.load('models/random_forest.pkl')
# Use the loaded model to make predictions
result = randomforest_from_joblib.predict(X_test)
acc = randomforest_from_joblib.score(X_test, y_test)
print(result)
print(acc)
unique, counts = np.unique(result, return_counts=True)
print(dict(zip(unique, counts)))

['male' 'male' 'male' ... 'female' 'female' 'female']
0.5371277299801456
{'female': 6994, 'male': 8116}


In [47]:
# NEURAL NETWORKS

from keras.models import model_from_json

# load json and create model
json_file = open('models/neural_network.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("models/neural_network.h5")
print("Loaded model from disk")

Loaded model from disk


In [48]:
# NEURAL NETWORKS

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

accuracy: 60.87%


In [49]:
# NEURAL NETWORKS

# evaluate the model
_, train_acc = loaded_model.evaluate(X_train, y_train, verbose=0)
_, test_acc = loaded_model.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))


Train: 0.949, Test: 0.609


In [34]:
#RANDOM FOREST AGE

youngCount = 0
oldCount = 0

directory = os.fsencode("temp/")
  
for file in os.listdir(directory):
    
    filename = os.fsdecode(file)
    test = pd.read_csv("temp/" + filename)

    test_X = test.drop(['age','gender'], axis=1)
    test_X  = preprocessing.scale(test_X)
    test_y = test['age']

    xgboost_from_joblib = joblib.load('models/rforest_age_tts_all.pkl')
    # Use the loaded model to make predictions

    result = xgboost_from_joblib.predict(test_X)
    # print(result)

    acc = xgboost_from_joblib.score(test_X, test_y)
    # print(acc)

    unique, counts = np.unique(result, return_counts=True)
    result = dict(zip(unique, counts))
    prediction = max(result, key=result.get)
    # print(result)
    # print(prediction)
    
    if prediction == '<40':
        youngCount += 1
    elif prediction == '>40':
        oldCount += 1
    else:
        print("error")

finalCount = {"<40": youngCount, ">40": oldCount}
print(finalCount)

{'<40': 88, '>40': 12}


In [36]:
#RANDOM FOREST GENDER

maleCount = 0
femaleCount = 0

directory = os.fsencode("temp/")
  
for file in os.listdir(directory):
    
    filename = os.fsdecode(file)
    test = pd.read_csv("temp/" + filename)

    test_X = test.drop(['age','gender'], axis=1)
    test_X  = preprocessing.scale(test_X)
    test_y = test['gender']

    xgboost_from_joblib = joblib.load('models/rforest_gender_tts_all.pkl')
    # Use the loaded model to make predictions

    result = xgboost_from_joblib.predict(test_X)
    # print(result)

    acc = xgboost_from_joblib.score(test_X, test_y)
    # print(acc)

    unique, counts = np.unique(result, return_counts=True)
    result = dict(zip(unique, counts))
    prediction = max(result, key=result.get)
    # print(result)
    # print(prediction)
    
    if prediction == 'male':
        maleCount += 1
    elif prediction == 'female':
        femaleCount += 1
    else:
        print("error")

finalCount = {"male": maleCount, "female": femaleCount}
print(finalCount)

{'male': 91, 'female': 9}


In [33]:
#XGBOOST AGE

youngCount = 0
oldCount = 0

directory = os.fsencode("temp/")
  
for file in os.listdir(directory):
    encodeAge = {'<40': 0, '>40': 1}
    encodeGender = {'male': 0, 'female': 1}

    filename = os.fsdecode(file)
    test = pd.read_csv("temp/" + filename)

    test_X = test.drop(['age','gender'], axis=1)
    # test_X['gender'] = test_X['gender'].map(encodeGender)
    test_X  = preprocessing.scale(test_X)
    test_y = test['age'].map(encodeAge)

    xgboost_from_joblib = joblib.load('models/xgboost_age_notts_234579_withgender.pkl')
    # Use the loaded model to make predictions

    result = xgboost_from_joblib.predict(test_X)
    # print(result)

    acc = xgboost_from_joblib.score(test_X, test_y)
    # print(acc)

    unique, counts = np.unique(result, return_counts=True)
    result = dict(zip(unique, counts))
    prediction = max(result, key=result.get)
    # print(result)
    # print(prediction)
    
    if prediction == 0:
        youngCount += 1
    elif prediction == 1:
        oldCount += 1
    else:
        print("error")

finalCount = {"<40": youngCount, ">40": oldCount}
print(finalCount)

{'<40': 4, '>40': 96}


In [41]:
#XGBOOST GENDER

maleCount = 0
femaleCount = 0

directory = os.fsencode("temp/")
  
for file in os.listdir(directory):
    encodeGender = {'male': 0, 'female': 1}
    
    filename = os.fsdecode(file)
    test = pd.read_csv("temp/" + filename)

    test_X = test.drop(['age','gender'], axis=1)
    test_X  = preprocessing.scale(test_X)
    test_y = test['gender'].map(encodeGender)

    xgboost_from_joblib = joblib.load('models/xgboost_gender_tts_all.pkl')
    # Use the loaded model to make predictions

    result = xgboost_from_joblib.predict(test_X)
    # print(result)

    acc = xgboost_from_joblib.score(test_X, test_y)
    # print(acc)

    unique, counts = np.unique(result, return_counts=True)
    result = dict(zip(unique, counts))
    prediction = max(result, key=result.get)
    # print(result)
    # print(prediction)
    
    if prediction == 0:
        maleCount += 1
    elif prediction == 1:
        femaleCount += 1
    else:
        print("error")

finalCount = {"male": maleCount, "female": femaleCount}
print(finalCount)

{'male': 98, 'female': 2}
